## Construindo a API do Twitter (HTTP Client)

In [1]:
import sys
import requests
import requests_oauthlib
import json
import tweepy
from datetime import date, timedelta

# Substitua os valores com as chaves da sua API criada em https://developer.twitter.com/en/apps
c_key = '1IzuR3F2cv8PTiqMEZMDroYW3'
c_secret = 'ksF3yvl0BaWEqdG1rMMwQpTIUu995cBCONp9t00ORWZlku66ds'
a_token = '184478772-BWnkYqSCARn4AsZTmVzlZlTy7isbTwgVrlDFfWGh'
a_secret = 'DOboqml5DlH0LCdDicEPErF6xODDxgl98xjs1SBTsaFnW'

last_month = date.today() - timedelta(days=30)

#conectando na API
auth = tweepy.OAuthHandler(c_key, c_secret)
auth.set_access_token(a_token, a_secret)
api = tweepy.API(auth)

def get_tweets_by_words(words):
    return tweepy.Cursor(api.search,
                         q = words,
                         since = last_month.strftime('%Y-%m-%d'),
                         until = date.today().strftime('%Y-%m-%d'),
                         include_entities = True,
                         lang="pt").items()

# Envia os tweets para o Spark
def send_tweets_to_spark(tweets, tcp_connection):
    for line in tweets:
        try:
            print(line._json)
            full_tweet = line._json
            tweet_text = full_tweet['text']
            #print("Tweet Text: " + tweet_text)
            print ("------------------------------------------")
            tcp_connection.send(bytes(tweet_text + '\n', 'utf-8'))
        except Exception as ex:
            ex = sys.exc_info()[0]
            print("Error enviando os tweetes para o Spark: %s" % ex)

## Configura e inicia o listener para obter os tweets

In [ ]:
import socket

_ip = "localhost"
_port = 9009
conn = None

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

s.bind((_ip, _port))
s.listen(1)
print("Conexão iniciada!")
print("Aguardando requisições...")
print("[Iniciar os serviços em 03_job_spark]")
conn, addr = s.accept()
print("Requisição recebida!")

print("Obtendo tweets...")
#tweets = get_tweets()
tweets = get_tweets_by_words(["chelsea"])

print("...")
print("Enviando tweets para o Spark...")
send_tweets_to_spark(tweets, conn)

Conexão iniciada!
Aguardando requisições...
[Iniciar os serviços em 03_job_spark]
Requisição recebida!
Obtendo tweets...
...
Enviando tweets para o Spark...
{'created_at': 'Wed Jun 03 23:58:14 +0000 2020', 'id': 1268331188579717126, 'id_str': '1268331188579717126', 'text': '@Vinidesanto @CuriosidadesPRL Shevchenko foi a pior contratação do Chelsea.', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'Vinidesanto', 'name': 'Vinícius Santos', 'id': 3006906845, 'id_str': '3006906845', 'indices': [0, 12]}, {'screen_name': 'CuriosidadesPRL', 'name': 'Curiosidades Premier League', 'id': 1227020047748673537, 'id_str': '1227020047748673537', 'indices': [13, 29]}], 'urls': []}, 'metadata': {'iso_language_code': 'pt', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': 1267944337222111232, 'in_reply_to_status_id_str': '1267944337222111232', 'in_reply_to